In [2]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8ijr0g2_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8ijr0g2_
  Resolved https://github.com/huggingface/transformers to commit 07bdbebb48a9fe1e748348e4e14ae0b4659e54c4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio


def feature_pipeline():

    common_voice = DatasetDict()

    common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
    common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)
    common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])


    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")
    processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

    def prepare_dataset(batch):
      # load and resample audio data from 48 to 16kHz
      audio = batch["audio"]

      # compute log-Mel input features from input audio array
      batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

      # encode target text to label ids
      batch["labels"] = tokenizer(batch["sentence"]).input_ids
      return batch

    common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

    return common_voice

In [6]:
# Run Feature Pipeline

common_voice = feature_pipeline()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You 

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 7308it [00:00, 74979.08it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5052it [00:00, 69262.53it/s]


...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Map:   0%|          | 0/12360 [00:00<?, ? examples/s]

Map:   0%|          | 0/5069 [00:00<?, ? examples/s]

In [9]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Save dataset to Google Drive
common_voice.save_to_disk(F"/content/gdrive/My Drive/ML/common_voice/")

Saving the dataset (0/24 shards):   0%|          | 0/12360 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/5069 [00:00<?, ? examples/s]

In [11]:
# Load from Google Drive
import os
print(os.listdir(F"/content/gdrive/My Drive/ML"))

['common_voice']


In [12]:
print(os.listdir(F"/content/gdrive/My Drive/ML/common_voice"))

['test', 'dataset_dict.json', 'train']


In [14]:
cc2 = DatasetDict.load_from_disk("/content/gdrive/My Drive/ML/common_voice")

In [15]:
cc2

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 12360
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5069
    })
})